## Apache Spark란

#### 1. 대용량 데이터 프로세싱을 위한 빠르고 범용적인 in-memory 기반(Disk도 가능)의 클러스터 컴퓨팅 엔진

- 배치, 대화형 쿼리, 스트리밍, ML등과 같은 다양한 Data 분석 작업 타입을 제공함.
- hadoop ecosystem에서는 각 작업타입마다 다른 도구를 사용해야 했지만, Spark는 하나로 통합된 환경을 제공한다는 것이 최대 장점.
- Big data system의 SOTA격인 hadoop과 엔진이 호환됨.
- Scala, Java, Python, R 등에서 코딩이 가능한 High-level API를 제공.
- Standalone scheduler, yarn, mesos : cluster 자원관리를 해주는 매니저. yarn이나 mesos와 달리, ss는 spark만 관리가 가능하다.

#### 2. Spark의 특징

- General execution graphs : DAG(Directed Acyclic Graph)의 특징을 가지고 있다. Job을 연쇄적 혹은 lazy하게 실행한다는 장점.

-----

## RDD (Resilient Distributed Dataset)

#### 1. RDD의 특징

- Dataset : 메모리나 디스크에 분산되어 저장된 변경 불가능한 데이터 객체들의 모음.
- Distributed : RDD에 있는 데이터는 클러스터에 자동적으로 분배되어 병렬연산을 수행한다. 이는 매니저의 역할로 수행됨.
- Resilient : 클러스터의 한 노드가 실패하더라도 다른 노드가 작업을 처리한다. Automatically rebuilt on failure의 구조이다.
- Immutable : RDD는 수정이 불가능하다. scala 기반 어플리케이션이기 때문에 함수형 프로그래밍의 장점중 하나(Immutable data)를 가져왔다. 변형을 통한 새로운 RDD를 사용해야 한다.
- Operation APIs : Transform의 역할로 map, filter, groupBy, join 등이 있다. 그리고 Actions의 역할로 count, collect 등이 있다.
- 가장 중요한 특징은 Lazy Evaluation이다. Action이 실행될때까지 Transform의 연산이 수행되지 않는다.
- Controllable Persistence : Cache를 RAM, Disk에 적재해놓고 사용하는 것이 가능하다. Action이 반복되는 경우, Cache를 이용하는 것이 연산량을 줄여준다.

#### 2. RDD 연산

- RDD는 기본적으로 context를 가지고 있다. 일반적으로 sc라는 변수로 context 객체를 생성.

###### example : RDD transformations ~ Action

In [ ]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext

val lines = sc.textfile(...)
// filter는 transformation, count는 action. action시에만 lazy evaluation이 시작된다.
lines.filter(x => x.contains("EROOR")).count()
// 여기서 lines는 RDD이고, filter와 count는 job master(명령 내리는 마스터 노드 혹은 프로그래머)가 실행하지만,
// (x => x.contains("EROOR")) 라는 실질적인 코드는 각각의 worker 노드가 실행하는 것임.

- worker 노드는 filter나 count같은 마스터가 하는 일을 수행하는 것이 아니고, 실직적인 코드만 수행함. filter나 count는 마스터가 실행하는 것.
- ex) 마스터는 x.contains를 찾으라고를 시킨다음에, worker가 보낸 결과들을 수집해서 count만 함.
- transformation, action은 master가 job을 시키는 액션과 같다고 보면 됨.

[LAB] 필기내용

- sc는 spark core에서 사용되는 context, spark는 sql에서 사용되는 context
- Flatmap -> 1개가 들어가서 N개가 나오는 map 작업
- (Ex : flatMap(l => l.split(“ “)) 에서 l이라는 1개의 라인이 들어가서 여러 개의 단어가 나옴.)
- Map -> 분리된 단어를 key, value 형태로 바꿔주는 작업
- Reducebykey : key로 reduce하여 모아주는 작업 (add. 네트워크(worker간)가 Shuffle 되면 stage가 바뀜.)
- Shuffle은 되도록 안하는게 좋지만, wc.saveAsTextFile 같은거로 마치 cache처럼 disk에 stage를 skip할 수 있는 Shuffle write를 해야 함.